# Evaluation of Cross-modal Query Suggestion methods on CroQS benchmark

In [1]:
device_str = "cuda:0"
device_str_llm = device_str

import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from lib.CroQS import CroQS

croQS_json_path = "./2024-10-07_15-35-43.json" #"./2024-07-21_14-52-15.json"
croQS = CroQS(croQS_json_path)
device = "cuda:0"
hdf5_file_path = "D:\\thesis-data\\hdf5-indexes\\coco_train_val_2017_image_embeddings.h5"
croQS.evaluation_init(hdf5_file_path, device)

No self.queries dict object loaded from disk, creating new empty dict


In [3]:
len(croQS.list_queries())

50

In [4]:
#def decap_centroid(q_0, embeddings_clip_space):
#    return ""
#
#methods_tuples = [
#    ("decap-centroid", decap_centroid),
#]

In [5]:
from lib.methodsLoader import get_qe_methods_dict, add_method_to_dict

qe_methods_names = [
    'initial-query',
    'clipcap-on-centroids', 
    'decap-on-centroids', 
    ##'clipcap-query_set-of-embeddings', 
    #'decap-on-representatives', 
    #'clipcap-on-representatives',
    #'clipcap-prompting-on-centroids',
    #'clipcap-prompting-on-representatives',
    ]

#qe_dict = get_qe_methods_dict(methods_keys=qe_methods_names, decap_qe=None, device=device_str)

In [6]:
def load_qe_dict(ir_system, load_group_cap : bool = False):
    from lib.methods.decap.Im2TxtProjection import Im2TxtProjector, ProjectionType
    from lib.methods.decap.DecapQueryExpansion import DeCapQueryExpansion
    from lib.methods.decap.decap import get_decap_model
    from lib.methods.groupcap.model import GroupCapLLM, LLMBasePrompts, ImageCaptioningMethods, LLMTypes

    decap_qe = DeCapQueryExpansion.load_object(device_str, 'coco')
    assert isinstance(decap_qe, DeCapQueryExpansion)
    decap_qe.im2txt.ir_system = ir_system
    qe_dict = get_qe_methods_dict(methods_keys=qe_methods_names, decap_qe=decap_qe, device=device_str)

    if load_group_cap:
        #group_cap_obj = GroupCapLLM(ImageCaptioningMethods.decap, decap_qe.get_generated_captions, LLMTypes.Mistral7b_IT, device_map=device_str, image_captioning_method_device=device_str)
        
        group_cap_obj = GroupCapLLM(ImageCaptioningMethods.decap, decap_qe.get_generated_captions, LLMTypes.LLama3_8b_IT, device_map=device_str_llm, image_captioning_method_device=device_str)
        
        
        prompts_dict = GroupCapLLM.get_prompts_dict()

        group_cap_obj.qe_prompt = prompts_dict['query-expansion']['qe-few-shot-1']
        #qe_dict = add_method_to_dict(qe_dict, 'groupcap-mistral7b-qe-few-shot-1', group_cap_obj.generate_expanded_query, wants_query=True, method_group_name="img-embeddings-whole-cluster")
        qe_dict = add_method_to_dict(qe_dict, 'groupcap-LLama3-8b-qe-few-shot-1', group_cap_obj.generate_expanded_query, wants_query=True, method_group_name="img-embeddings-whole-cluster")

        #group_cap_obj.base_prompt = prompts_dict['group-captioning']['cap-prompt1']
        #group_cap_obj.qe_prompt = prompts_dict['query-expansion']['qe-prompt2']
        #qe_dict = add_method_to_dict(qe_dict, 'groupcap-mistral7b-cap-prompt1', group_cap_obj.generate_caption, method_group_name="img-embeddings-whole-cluster")
        #qe_dict = add_method_to_dict(qe_dict, 'groupcap-mistral7b-qe-prompt2', group_cap_obj.generate_expanded_query, wants_query=True, method_group_name="img-embeddings-whole-cluster")
    
    return qe_dict

qe_dict = load_qe_dict(croQS.ir_system, False)

In [7]:
croQS.get_query_suggestions("dog running", "0")

{'initial-query': 'dog running',
 'human': 'dog running catching a frisbee',
 'clipcap-on-centroids': 'A dog is jumping to catch a frisbee.'}

In [8]:
croQS.load_query_suggestions("dog running", "0", qe_dict)

Loading already existing qs for the method 'initial-query'troids'Computing scores for cluster '0' of query = 'dog running'


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


({'initial-query': 'dog running',
  'human': 'dog running catching a frisbee',
  'clipcap-on-centroids': 'A dog is jumping to catch a frisbee.',
  'decap-on-centroids': 'a dog running to catch a frisbee in its mouth . '},
 True)

In [9]:
ret_df = croQS.get_query_suggestions_dataframe_all_queries(qe_dict)

ret_df

Loading already existing qs for the method 'initial-query'oids's'Computing scores for cluster '2' of query = 'dog running'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '0' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '1' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '2' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '3' of query = 'person reading'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '0' of query = 'laptop near'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '1' of query = 'laptop near'
Loading already existing qs for the method 'initial-query'Computing scores for cluster '2' of query = 'laptop near'
Loading already existing qs for the method 'initial-q

In [8]:
#croQS.evaluate(methods_tuples)

In [9]:
qq = 'person at the phone'
cc = '3'

croQS._list_clusters_labels(qq)

['0', '1', '2', '3']

In [10]:
croQS.suggested_queries[qq][cc]

{'suggested-queries': {'initial-query': {'query': 'person at the phone',
   'scores': {'jaccard-q': 1.0,
    'clip-similarity-cluster': 0.294,
    'clip-similarity-q': 1.0,
    'NDCG@10': 0.23365080824310536,
    'MAP': 0.2868582289545055,
    'Recall-Open-Set-@100': 0.5128205128205128,
    'Recall-Closed-Set': 0.28205128205128205}},
  'human': {'query': 'woman talking on mobile phone',
   'scores': {'jaccard-q': 0.2,
    'clip-similarity-cluster': 0.317,
    'clip-similarity-q': 0.877,
    'NDCG@10': 0.39639187290150935,
    'MAP': 0.3438731991303363,
    'Recall-Open-Set-@100': 0.5128205128205128,
    'Recall-Closed-Set': 0.5897435897435898}}}}

In [11]:
for idx, el in enumerate(croQS.list_queries()):
    #print(f"\r{el}", end="")
    if el == qq:
        print(idx, "ZP")
    if len(croQS._list_clusters_labels(el)) == 0:
        print(f"the q '{el}' ({idx=}) has no cluster")

29 ZP
